In [1]:
from flask import Flask
from flask import request, jsonify

from transformers import BertTokenizer, BertForSequenceClassification
import torch
import re

import pandas as pd
import numpy as np
import pickle

/home/ubuntu/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import nltk
from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import torch.nn.functional as nnf

# ...


#load models
model = torch.load('bert_opossum_best_0.6753.pt')
tokenizer = BertTokenizer.from_pretrained('sberbank-ai/ruBert-base')
device = 'cuda'
model.to(device)
pass

In [3]:
#tokenize, predict, text preprocessing functions

def tokenize_text(text):
    return tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        )

def predict(df):
    batch = pd.DataFrame.from_records(df['text'].apply(tokenize_text).values)
    input_ids = [a.numpy()[0] for a in batch.iloc[:,1].values]
    input_ids = torch.from_numpy(np.array([a for a in input_ids],dtype=int)).to(device)
    attention_mask =  [a.numpy()[0] for a in batch.iloc[:,0].values]
    attention_mask = torch.from_numpy(np.array([a for a in attention_mask],dtype=int)).to(device)
    
    outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
    )
    
    prediction = outputs.logits

    prob = nnf.softmax(prediction , dim=1)

    top_p, top_class = prob.topk(1, dim = 1)
    
    df['score'] = (top_p.cpu().detach().numpy().reshape(-1)*100).astype(int)
    df['type']=top_class.cpu().detach().numpy().reshape(-1)

russian_stopwords = stopwords.words("russian")

russian_stopwords.append('российской')
russian_stopwords.append('федерации')
russian_stopwords.append('федерального')
russian_stopwords.append('настоящих')
russian_stopwords.append('соответствии')
russian_stopwords.append('также')
russian_stopwords.append('рф')
russian_stopwords.append('ред')

russian_stopwords.append('субсидии')
russian_stopwords.append('предоставления')


def lowercase(text):
    return str(text).lower()

def clean_symb(text):
    return re.sub(r'[^\w]', ' ', text)

def clear_token(text):
    return word_tokenize(text)

def clean_stopwords(token):
    return ' '.join([i for i in token.split(' ') if i not in russian_stopwords])

In [4]:
#test json data

json = {'1': '1. {2} Настоящие Правила устанавливают цели, условия и порядок предоставления субсидии из федерального бюджета Фонду "Центр стратегических разработок" (далее - Фонд) в целях оценки эффектов от реализации инвестиционных проектов в сфере транспорта в рамках государственной программы Российской Федерации "Экономическое развитие и инновационная экономика" (далее - субсидия). {2}', '2': '2. {3} Предоставление субсидии осуществляется в пределах лимитов бюджетных обязательств, доведенных в установленном порядке до Министерства экономического развития Российской Федерации как получателя средств федерального бюджета на цели, указанные в пункте 1 настоящих Правил. {3}', '3': '3. {24} Субсидия предоставляется на основании соглашения о предоставлении субсидии, заключаемого между Министерством экономического развития Российской Федерации и Фондом (далее - соглашение о предоставлении субсидии). {24}', '4': '4. {24} Соглашение о предоставлении субсидии содержит в том числе: {24}', '5': '5. {24} Соглашение о предоставлении субсидии и дополнительные соглашения к нему, предусматривающие внесение изменений, или дополнительное соглашение о расторжении соглашения о предоставлении субсидии заключаются в государственной интегрированной информационной системе управления общественными финансами "Электронный бюджет" в соответствии с типовой формой, установленной Министерством финансов Российской Федерации.{24}', '6': '6. {4} Субсидия предоставляется на финансовое обеспечение затрат, связанных с достижением целей, указанных в пункте 1 настоящих Правил, в том числе понесенных Фондом в текущем финансовом году до заключения соглашения о предоставлении субсидии (при наличии документов, подтверждающих фактически произведенные затраты), в размере, определяемом по формуле: {4}', '7': '7. {22} Размер субсидии (Рсуб) определяется в пределах лимитов бюджетных обязательств, утвержденных и доведенных в установленном порядке до Министерства экономического развития Российской Федерации как получателя средств федерального бюджета на цели, указанные в пункте 1 настоящих Правил. {22}', '8': '8. {11} Фонд по состоянию на 1-е число месяца, предшествующего месяцу, в котором заключается соглашение о предоставлении субсидии, должен соответствовать следующим требованиям: {11}', '9': '9. {19} Для заключения соглашения о предоставлении субсидии Фонд представляет в Министерство экономического развития Российской Федерации документы, подписанные руководителем Фонда (иным уполномоченным лицом), подтверждающие соответствие Фонда каждому из требований, предусмотренных пунктом 8 настоящих Правил. {19}'}

In [5]:
#test prediction from data

content =json
df = pd.DataFrame.from_dict(content,orient='index',columns=['text'])
df['text'] = df.text.apply(lowercase)
df['text'] = df.text.apply(clean_symb)
df['text'] = df.text.apply(lambda x:''.join([a for a in x if not a.isdigit()]))
df['text'] = df.text.apply(lambda x:' '.join([a for a in x.split(' ') if len(a)>1]))
df['text'] = df.text.apply(clean_stopwords)
predict(df)
df['type']+=1

print(df[['type','score']].T.to_dict(orient='list'))

{'1': [2, 99], '2': [4, 46], '3': [24, 90], '4': [24, 99], '5': [24, 96], '6': [4, 91], '7': [22, 99], '8': [11, 86], '9': [19, 99]}


In [ ]:
#flask api

app = Flask(__name__)

@app.route('/api', methods=['GET', 'POST'])
def info():
    content = request.json
    df = pd.DataFrame.from_dict(content,orient='index',columns=['text'])
    print(df.shape)
    df['text'] = df.text.apply(lowercase)
    df['text'] = df.text.apply(clean_symb)
    df['text'] = df.text.apply(lambda x:''.join([a for a in x if not a.isdigit()]))
    df['text'] = df.text.apply(lambda x:' '.join([a for a in x.split(' ') if len(a)>1]))
    df['text'] = df.text.apply(clean_stopwords)
    if df.shape[0]>0:
        predict(df)
        df['type']+=1

    return df[['type','score']].T.to_dict(orient='list')

if __name__ == '__main__':
    app.run(host='172.19.0.178',port=5000)